# This is just a first tests. Don't judge me

# Importing and processing raw data

In [1]:
import pandas as pd
from sklearn import metrics

# read in csv data
matches = pd.read_csv("AI Match Results 150years_extended_20221205.csv")
#matches = pd.read_csv("with_add_Features.csv")
print("matches.shape: ", matches.shape)
#print("matches.dtypes: \n", matches.dtypes)
matches.head()

matches.shape:  (44109, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


## Creating new overall features

- calculating the strength of each team using ELO https://en.wikipedia.org/wiki/World_Football_Elo_Ratings
- calculating goals achived (fGoals) and goals against (aGoals) for eacht team overall


In [59]:
# create dict for tournament type -> moved to K_val.py
from K_val import K

def calcNewELO( R_old, R_opp, tournament_type, own_score, opp_score, is_home_team, neutral):
    # R_new = R_old + K * G * (W - W_e)
    #   R_new = The new team rating
    #   R_old = The old team rating
    #   K = Weight index regarding the tournament of the match
    #       World Cup, Olympic Games (1908–1980) 	60
    #       Continental championship and intercontinental tournaments 	50
    #       World Cup and Continental qualifiers and major tournaments 	40
    #       All other tournaments 	30
    #       Friendly matches 	30
    #   G = A number from the index of goal differences
    #   W = The result of the match
    #   W_e = The expected result
    G=1
    W=1

    # set G according to http://www.eloratings.net/about
    if (own_score >= (opp_score+4)):
        N = own_score-opp_score
        G = 1 + 3/4 + (N-3)/8
    elif (own_score > opp_score+3):
        G = 1.5

    # dr equals the difference in ratings plus 100 points for a team playing at home. 
    if (neutral):
        dr = R_old - R_opp
    elif (is_home_team):
        dr = R_old + 100 - R_opp
    else:
        dr = R_old - (R_opp +100)
    
    # We = 1 / ( (10^(-dr/400)) + 1)
    We = 1 / ( pow(10, (-dr/400)) + 1)

    if (own_score > opp_score):
        W = 1
    elif (own_score ==  opp_score):
        W = 0.5
    else:
        W = 0

    R_new = R_old + K[tournament_type] * G * (W - We)
    
    return R_new

# MAIN

# update matches data with ELO, fGoals, aGoals of each team
matches["ELO_home"] = 0.0
matches["ELO_away"] = 0.0
matches["fGoals_home"] = 0
matches["aGoals_home"] = 0
matches["fGoals_away"] = 0
matches["aGoals_away"] = 0
matches["fGoals_p_match_home"] = 0.0
matches["aGoals_p_match_home"] = 0.0
matches["fGoals_p_match_away"] = 0.0
matches["aGoals_p_match_away"] = 0.0
matches["num_matches_home"] = 0
matches["num_matches_away"] = 0
matches["faGoals_ratio_home"] = 0.0
matches["faGoals_ratio_away"] = 0.0

# make a list of all teams using "home_team" and "away_team"
uteams = (matches["home_team"]._append( matches["away_team"] )).unique()

# initialie ELO with 1000
team_stats = pd.DataFrame({
                   "index": uteams,
                   "name": uteams,
                   "ELO": 1000.0,
                   "fGoals": 1, # every team starts with fGoals/aGoals score of 1 otherwise the ration would be Inf at the beginning
                   "aGoals": 1, # since the overall goals will be a high value this offset of 1 should influence much
                   "num_matches": 0,
                   "fGoals_p_match": 0.0,
                   "aGoals_p_match": 0.0})
team_stats.set_index("index", inplace = True)

# generate a team code for each team in team_stats dataframe
team_stats["team_code"] = team_stats["name"].astype("category").cat.codes


for ind in matches.index:
    # store home and away team name
    ht = matches["home_team"][ind]
    at = matches["away_team"][ind]

    ############################ Team Codes ##############################
    matches.at[ind, "home_team_code"] = team_stats.at[ht, "team_code"]
    matches.at[ind, "away_team_code"] = team_stats.at[at, "team_code"]
     
    ################################ ELO #################################
    
    # get ELO from team_stats DataFrame (before match) and write into matches DataFrame 
    matches.at[ind, "ELO_home"] = team_stats.at[ht, "ELO"]
    matches.at[ind, "ELO_away"] = team_stats.at[at, "ELO"]
    # calc new ELO
    Rn_home = calcNewELO(matches["ELO_home"][ind], matches["ELO_away"][ind], matches["tournament"][ind], matches["home_score"][ind], matches["away_score"][ind], not matches["neutral"][ind], matches["neutral"][ind])
    Rn_away = calcNewELO(matches["ELO_away"][ind], matches["ELO_home"][ind], matches["tournament"][ind], matches["away_score"][ind], matches["home_score"][ind], False, matches["neutral"][ind])
    # Update ELO after match
    team_stats.at[ht, "ELO"] = float(Rn_home)
    team_stats.at[at, "ELO"] = float(Rn_away)

    ######################### For/Against Goals #########################
    # store home and away team goals before match
    matches.at[ind, "fGoals_home"] = team_stats.at[ht, "fGoals"]
    matches.at[ind, "aGoals_home"] = team_stats.at[ht, "aGoals"]
    matches.at[ind, "fGoals_away"] = team_stats.at[at, "fGoals"]
    matches.at[ind, "aGoals_away"] = team_stats.at[at, "aGoals"]
    # update foa and against goals
    team_stats.at[ht, "fGoals"] = team_stats.at[ht, "fGoals"] + matches["home_score"][ind]
    team_stats.at[at, "fGoals"] = team_stats.at[at, "fGoals"] + matches["away_score"][ind]
    team_stats.at[ht, "aGoals"] = team_stats.at[ht, "aGoals"] + matches["away_score"][ind]
    team_stats.at[at, "aGoals"] = team_stats.at[at, "aGoals"] + matches["home_score"][ind]

    matches.at[ind, "faGoals_ratio_home"] = team_stats.at[ht, "fGoals"] / team_stats.at[ht, "aGoals"]
    matches.at[ind, "faGoals_ratio_away"] = team_stats.at[at, "fGoals"] / team_stats.at[at, "aGoals"]

    ##################### Average Goals per match ########################
    # store home and away team goals before match
    matches.at[ind, "num_matches_home"] = team_stats.at[ht, "num_matches"]
    matches.at[ind, "num_matches_away"] = team_stats.at[at, "num_matches"]
    matches.at[ind, "fGoals_p_match_home"] = team_stats.at[ht, "fGoals_p_match"]
    matches.at[ind, "aGoals_p_match_home"] = team_stats.at[ht, "aGoals_p_match"]
    matches.at[ind, "fGoals_p_match_away"] = team_stats.at[at, "fGoals_p_match"]
    matches.at[ind, "aGoals_p_match_away"] = team_stats.at[at, "aGoals_p_match"]
    
    # update after match
    ht_num_matches = team_stats.at[ht, "num_matches"]+1
    at_num_matches = team_stats.at[at, "num_matches"]+1
    team_stats.at[ht, "num_matches"] = ht_num_matches
    team_stats.at[at, "num_matches"] = at_num_matches

    team_stats.at[ht, "fGoals_p_match"] = team_stats.at[ht, "fGoals"] / ht_num_matches
    team_stats.at[ht, "aGoals_p_match"] = team_stats.at[ht, "aGoals"] / ht_num_matches
    team_stats.at[at, "fGoals_p_match"] = team_stats.at[at, "fGoals"] / at_num_matches
    team_stats.at[at, "aGoals_p_match"] = team_stats.at[at, "aGoals"] / at_num_matches


    
matches.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,ELO_home,...,t2t_fGoals_home,t2t_fGoals_away,t2t_fGoals_p_match_home,t2t_fGoals_p_match_away,t2t_num_wins_home,t2t_num_wins_away,t2t_num_draw,t2t_p_wins_home,t2t_p_wins_away,t2t_p_draw
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1000.000000,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1002.801300,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,990.147921,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1002.122893,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1000.790631,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False,1007.947444,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False,992.052556,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False,993.010717,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False,1044.776295,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False,1055.945242,...,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0


## Creating team-to-team features

- calculating goals achived (fGoals) and goals against (aGoals) and match statistics (win/lose/draw) for eacht team against the specific opponent

In [2]:
# this will take a while....
import datetime as dt

def get_t2t_data(ht, at, date):
    t2t_fGoals_home = 0
    t2t_fGoals_away = 0
    t2t_fGoals_p_match_home = 0.0
    t2t_fGoals_p_match_away = 0.0
    t2t_num_wins_home = 0
    t2t_num_wins_away = 0
    t2t_num_draw = 0
    t2t_p_wins_home = 0.0
    t2t_p_wins_away = 0.0
    t2t_p_draw = 0.0

    # get two subset of matches where the two teams played against each other home-vs-away and away-vs-home before the current match
    t2t_matches1 = matches.loc[(matches["home_team"] == ht) & (matches["away_team"] == at) & (matches["date"] < date)]
    t2t_matches2 = matches.loc[(matches["home_team"] == at) & (matches["away_team"] == ht) & (matches["date"] < date)]
    # invert home_team and away_team for the second subset
    idx = t2t_matches2.index
    t2t_matches2.loc[idx,["home_team","away_team","home_score","away_score"]] = t2t_matches2.loc[idx,["away_team","home_team","away_score","home_score"]].values
    # append both dataframes to one
    t2t_matches = t2t_matches1._append(t2t_matches2)

    # exctract features
    num_matches = t2t_matches.shape[0]

    if (num_matches > 0):
        # number of goals ooverall for each team
        t2t_fGoals_home = t2t_matches["home_score"].sum()
        t2t_fGoals_away = t2t_matches["away_score"].sum()
        # number of goals per match
        t2t_fGoals_p_match_home = t2t_fGoals_home/num_matches
        t2t_fGoals_p_match_away = t2t_fGoals_away/num_matches
        # number of wins/lose/draw
        t2t_num_wins_home = t2t_matches.loc[t2t_matches["home_score"] > t2t_matches["away_score"]].shape[0]
        t2t_num_wins_away = t2t_matches.loc[t2t_matches["home_score"] < t2t_matches["away_score"]].shape[0]
        t2t_num_draw = t2t_matches.loc[t2t_matches["home_score"] == t2t_matches["away_score"]].shape[0]
        # percetage of wins/lose/draw
        t2t_p_wins_home = t2t_num_wins_home/num_matches
        t2t_p_wins_away = t2t_num_wins_away/num_matches
        t2t_p_draw = t2t_num_draw/num_matches

    return [t2t_fGoals_home, 
            t2t_fGoals_away, 
            t2t_fGoals_p_match_home,
            t2t_fGoals_p_match_away,
            t2t_num_wins_home,
            t2t_num_wins_away,
            t2t_num_draw,
            t2t_p_wins_home,
            t2t_p_wins_away,
            t2t_p_draw]


# Save the current time
t = dt.datetime.now()

#convert date into usable format
matches["date"] = pd.to_datetime(matches["date"])

matches["t2t_fGoals_home"] = 0
matches["t2t_fGoals_away"] = 0
matches["t2t_fGoals_p_match_home"] = 0.0
matches["t2t_fGoals_p_match_away"] = 0.0
matches["t2t_num_wins_home"] = 0
matches["t2t_num_wins_away"] = 0
matches["t2t_num_draw"] = 0
matches["t2t_p_wins_home"] = 0.0
matches["t2t_p_wins_away"] = 0.0
matches["t2t_p_draw"] = 0.0

# go through matches
# for ind in matches.index:
#     # new team to team features
#     ht = matches["home_team"][ind]
#     at = matches["away_team"][ind]
#     date = matches["date"][ind]

#     #print(ht, at, da)
#     matches.at[ind, [   "t2t_fGoals_home",
#                         "t2t_fGoals_away",
#                         "t2t_fGoals_p_match_home",
#                         "t2t_fGoals_p_match_away",
#                         "t2t_num_wins_home",
#                         "t2t_num_wins_away",
#                         "t2t_num_draw",
#                         "t2t_p_wins_home",
#                         "t2t_p_wins_away",
#                         "t2t_p_draw"]] = get_t2t_data(ht, at, date)


#     delta = dt.datetime.now()-t
#     if delta.seconds >= 30:
#         # print progress every minute
#         #print("Currently at match at: ", da)
#         print("Progress: {:.1f}".format(ind*100/matches.shape[0]), "%")
#         # Update 't' variable to new time
#         t = dt.datetime.now()

# matches[[   "t2t_fGoals_home",
#             "t2t_fGoals_away",
#             "t2t_fGoals_p_match_home",
#             "t2t_fGoals_p_match_away",
#             "t2t_num_wins_home",
#             "t2t_num_wins_away",
#             "t2t_num_draw",
#             "t2t_p_wins_home",
#             "t2t_p_wins_away",
#             "t2t_p_draw"]] = matches.apply(lambda row: get_t2t_data(row['home_team'], row['away_team'], row['date']), axis=1, result_type='expand')
# matches.head(20)



In [ ]:
matches.head()

In [ ]:
# store data to csv
matches.to_csv("matches_with_new_features.csv")
team_stats.to_csv("team_stats.csv")  

# Skip all above if no new features and import pre-generated sheet

In [3]:
import pandas as pd
from sklearn import metrics

# read in csv data
matches = pd.read_csv("matches_with_new_features.csv")
team_stats = pd.read_csv("team_stats.csv")
team_stats.set_index("index", inplace = True)
print("matches.shape: ", matches.shape)
#print("matches.dtypes: \n", matches.dtypes)
team_stats.head()

matches.shape:  (44109, 37)


,name,ELO,fGoals,aGoals,num_matches,fGoals_p_match,aGoals_p_match,team_code
index,,,,,,,,
Scotland,Scotland,1413.571029,1395,996,813,1.715867,1.225092,241
England,England,1649.420210,2292,1017,1047,2.189112,0.971347,85
Wales,Wales,1385.198241,859,1122,692,1.241329,1.621387,300
Northern Ireland,Northern Ireland,1206.894299,703,1255,673,1.044577,1.864785,198
United States,United States,1524.948033,1078,945,728,1.480769,1.298077,291


### preprocess data

In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np

#convert date into usable format
matches["date"] = pd.to_datetime(matches["date"])

################ coding type<object> data ################
# convert object columns into integers

# categorize tournament
matches["tournament_code"] = matches["tournament"].astype("category").cat.codes

################ scale data ################


## TODo: Scale train set only and apply same scale to testset after 
goal_scaler = StandardScaler()
goal_pm_scaler = StandardScaler()
ELO_scaler = StandardScaler()
tournament_scaler = StandardScaler()
team_scaler = StandardScaler()
num_win_scaler = StandardScaler()

matches[[   "fGoals_home_sc", 
            "aGoals_home_sc", 
            "fGoals_away_sc", 
            "aGoals_away_sc",
            "t2t_fGoals_home_sc",
            "t2t_fGoals_away_sc" ]] = goal_scaler.fit_transform(matches[["fGoals_home", "aGoals_home", "fGoals_away", "aGoals_away", "t2t_fGoals_home", "t2t_fGoals_away"]])

matches[[   "fGoals_p_match_home_sc", 
            "aGoals_p_match_home_sc", 
            "fGoals_p_match_away_sc", 
            "aGoals_p_match_away_sc",
            "t2t_fGoals_p_match_home_sc",
            "t2t_fGoals_p_match_away_sc" ]] = goal_pm_scaler.fit_transform(matches[["fGoals_p_match_home", "aGoals_p_match_home", "fGoals_p_match_away", "aGoals_p_match_away","t2t_fGoals_p_match_home", "t2t_fGoals_p_match_away"]])

matches[[   "ELO_home_sc", 
            "ELO_away_sc"]] = ELO_scaler.fit_transform(matches[["ELO_home", "ELO_away"]])

matches[[   "tournament_code_sc" ]] = tournament_scaler.fit_transform(matches[["tournament_code" ]])

matches[[   "home_team_code_sc", 
            "away_team_code_sc"]] = team_scaler.fit_transform(matches[["home_team_code", "away_team_code"]])

matches[[   "t2t_num_wins_home_sc", 
            "t2t_num_wins_away_sc",
            "t2t_num_draw_sc" ]] = num_win_scaler.fit_transform(matches[["t2t_num_wins_home", "t2t_num_wins_away", "t2t_num_draw"]])
            


################ calculate desired output ################
# convert result into boolean statements
matches["home_win"] = (matches["home_score"] > matches["away_score"])
matches["away_win"] = (matches["home_score"] < matches["away_score"])
matches["draw"] = (matches["home_score"] == matches["away_score"])

matches["result"] = "home_win"
matches.loc[matches["home_score"] < matches["away_score"], "result"] = "away_win"
matches.loc[matches["home_score"] == matches["away_score"], "result"] = "draw"
matches["result_code"] = matches["result"].astype("category").cat.codes
matches["result_cat"] = matches["result"].astype("category")

matches["goal_diff"] = abs(matches["home_score"] - matches["away_score"])
matches["loser_score"] = matches[["home_score", "away_score"]].min(axis=1)


################ reduce dataset ################

#drop old data before a certain date
red_matches  = matches.drop(matches[matches["date"] < "1993-01-01"].index)

# drop first third of data since ELO algortithm itself needs time to be adjusted:
# a = int(0.33*matches.shape[0])
# red_matches = matches.iloc[a:, :]   # around 2.5% precission improvent

################ split train and test data ################

#use 80% for training and the remaining for testing
train_raw = red_matches.sample(frac = 0.8)
test  = red_matches.drop(train_raw.index)

# unskew train data
train0 = train_raw.loc[train_raw["result_code"] == 0]
train1 = train_raw.loc[train_raw["result_code"] == 1]
train2 = train_raw.loc[train_raw["result_code"] == 2]

num_to_keep_of_every_class = int((train0.shape[0]+train2.shape[0]+train2.shape[0])/3) # keep average of all classes
train0 = train0.sample(min(train0.shape[0], num_to_keep_of_every_class))
train1 = train1.sample(min(train1.shape[0], num_to_keep_of_every_class))
train2 = train2.sample(min(train2.shape[0], num_to_keep_of_every_class))

train_unskew_result = train0._append(train1)._append(train2).sample(frac = 1) # concate everything and mix

print("Full set:            ", matches.shape[0])
print("Reduced set:         ", red_matches.shape[0])
print("Train data raw:      ", train_raw.shape[0])
print("Train data unskew:   ", train_unskew_result.shape[0])
#print("Train data skew val: ", train_unskew_result[["date", "result_code"]].skew(axis=0)[0])
print("Test data:           ", test.shape[0])

#train.head()


Full set:             44109
Reduced set:          26411
Train data raw:       21129
Train data unskew:    19654
Test data:            5282


In [5]:
# unskewing goal_diff data

num_to_keep_of_every_class = int(np.mean(train_raw.groupby(['goal_diff']).size()))

#create unique list of names
UniqueKeys = train_raw.goal_diff.unique()

train_unskew_goals = pd.DataFrame(columns = train_raw.columns)

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame() for elem in UniqueKeys}

for key in UniqueKeys:
    DataFrameDict = train_raw[train_raw["goal_diff"] == key]
    DataFrameDict= DataFrameDict.sample(min(DataFrameDict.shape[0], num_to_keep_of_every_class))
    train_unskew_goals = train_unskew_goals._append(DataFrameDict)


train_unskew_goals.shape[0]

5639

# Match result classification

#### Baseline: Decide exclusively based on ELO

In [12]:
#bl_predictions = test["ELO_home"] > test["ELO_away"]

bl_predictions = test.copy()

bl_predictions["ELO_result"] = "draw"
bl_predictions.loc[test["ELO_home"] < test["ELO_away"]-20, "ELO_result"] = "away_win"
bl_predictions.loc[test["ELO_home"] > test["ELO_away"]+20, "ELO_result"] = "home_win"


# Scores
#acc_score = metrics.accuracy_score(test[targets], bl_predictions)
#prec_score = metrics.precision_score(test[targets], bl_predictions)

#print("Accuracy:   ", acc_score)
#print("Precission: ", prec_score)

BL_Elo_Report = metrics.classification_report(bl_predictions["result"], bl_predictions["ELO_result"])
print(BL_Elo_Report)
#bl_predictions[["ELO_home", "ELO_away", "result", "ELO_result"]].head()

#bl_predictions.to_excel("BL.xlsx")  

              precision    recall  f1-score   support

    away_win       0.45      0.69      0.55      1457
        draw       0.25      0.09      0.13      1287
    home_win       0.67      0.68      0.67      2538

    accuracy                           0.54      5282
   macro avg       0.46      0.49      0.45      5282
weighted avg       0.51      0.54      0.51      5282



### MLP

In [ ]:
# fitting
from sklearn.neural_network import MLPRegressor

train = train_unskew_result

mlp_features = ["home_team_code_sc", 
                "away_team_code_sc", 
                "ELO_home_sc", 
                "ELO_away_sc", 
                "neutral", 
                "fGoals_p_match_home_sc", 
                "aGoals_p_match_home_sc", 
                "fGoals_p_match_away_sc", 
                "aGoals_p_match_away_sc", 
                "t2t_p_wins_home", 
                "t2t_p_wins_away",
                "t2t_p_draw"]
mlp_targets = ["home_win", "draw", "away_win"]#, "goal_diff", "loser_score"]

mlp = MLPRegressor(solver="lbfgs", alpha=1e-5, hidden_layer_sizes=(50, 20), random_state=1)

mlp.fit(train[mlp_features], train[mlp_targets])

# make predictions
mlp_predictions_raw = mlp.predict(test[mlp_features])
mlp_predictions = pd.DataFrame(mlp_predictions_raw, columns = mlp_targets)
# tranlate result probs into result
mlp_predictions["result"] = mlp_predictions[["home_win", "away_win", "draw"]].idxmax(axis=1)

# generate report
MLP_Report = metrics.classification_report(test["result"], mlp_predictions["result"])
print(MLP_Report)


In [ ]:
# visualize prediction (not needed)
mlp_predictions.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
a = pd.concat([test[["home_team", "away_team", "ELO_home", "ELO_away", "neutral", "fGoals_home", "aGoals_home", "fGoals_away", "aGoals_away", "result"]], mlp_predictions], axis=1, keys=["Train Data", "Prediction"])
a.head(10)

### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train = train_unskew_result

# fitting
rf = RandomForestClassifier(n_estimators=100, min_samples_split=5, random_state=1)

rf_features = [ "home_team_code_sc", 
                "away_team_code_sc", 
                "ELO_home_sc", 
                "ELO_away_sc", 
                "neutral", 
                "fGoals_p_match_home_sc", 
                "aGoals_p_match_home_sc", 
                "fGoals_p_match_away_sc", 
                "aGoals_p_match_away_sc", 
                "t2t_p_wins_home", 
                "t2t_p_wins_away",
                "t2t_p_draw"]
rf_targets = ["result"]

rf.fit(train[rf_features], train[rf_targets])

# make predictions
rf_predictions = rf.predict(test[rf_features])

# generate report
RF_Report = metrics.classification_report(test["result"], rf_predictions)
print(RF_Report)

In [ ]:
# visualize prediction (not needed)
a = test.copy()
a["prediction"] = rf_predictions

a[["home_team", "away_team", "ELO_home", "ELO_away", "neutral", "fGoals_home", "aGoals_home", "fGoals_away", "aGoals_away", "result", "prediction"]].head(10)

### XGBoost

In [6]:
import xgboost as xgb

train = train_unskew_result

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 200)

xg_features = [ "home_team_code_sc", 
                "away_team_code_sc", 
                "ELO_home_sc", 
                "ELO_away_sc", 
                "neutral", 
                "fGoals_p_match_home_sc", 
                "aGoals_p_match_home_sc", 
                "fGoals_p_match_away_sc", 
                "aGoals_p_match_away_sc", 
                "t2t_p_wins_home", 
                "t2t_p_wins_away",
                "t2t_p_draw"]
xg_targets = ["home_win", "draw", "away_win"]

xg_reg.fit(train[xg_features], train[xg_targets])

# make predictions
xg_predictions_raw =  xg_reg.predict(test[xg_features])
xg_predictions = pd.DataFrame(xg_predictions_raw, columns = xg_targets)
# tranlate result probs into result
xg_predictions["result"] = xg_predictions[["home_win", "away_win", "draw"]].idxmax(axis=1)

# generate report
XG_Report = metrics.classification_report(test["result"], xg_predictions["result"])
print(XG_Report)

c:\toolbase\python\3.11.4.2\Lib\site-packages\xgboost\core.py:160: UserWarning: [08:25:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

    away_win       0.52      0.60      0.56      1487
        draw       0.33      0.09      0.14      1256
    home_win       0.62      0.79      0.70      2539

    accuracy                           0.57      5282
   macro avg       0.49      0.49      0.47      5282
weighted avg       0.53      0.57      0.53      5282



In [10]:
# visualize prediction (not needed)
xg_predictions.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
a = pd.concat([test[["home_team", "away_team", "ELO_home", "ELO_away", "neutral", "fGoals_home", "aGoals_home", "fGoals_away", "aGoals_away", "result"]], xg_predictions], axis=1, keys=["Train Data", "Prediction"])
a.head(10)

Train Data                                                              \
       home_team     away_team     ELO_home     ELO_away neutral fGoals_home   
0          Gabon  Burkina Faso  1123.572383  1082.654529   False         117   
1         Kuwait       Lebanon  1152.753141   956.913306   False         340   
2   Sierra Leone          Togo  1124.190310  1040.581562    True         127   
3         Uganda      Tanzania  1125.830547   967.326526   False         563   
4   South Africa       Nigeria  1154.135644  1338.097182   False          91   
5       Botswana   Ivory Coast   874.408752  1262.039697   False          25   
6           Togo      Zimbabwe  1044.123140  1162.512984   False         169   
7  Guinea-Bissau        Guinea   984.975654  1088.836903   False          70   
8          Italy        Mexico  1479.033528  1380.371907   False         862   
9      Venezuela          Peru   895.606049  1096.102453   False          76   

                                                Prediction            \
  aGoals_home fGoals_away aGoals_away    result   home_win      draw   
0         140         125         172      draw   0.576545  0.257931   
1         224          43          96  home_win   0.682081  0.222097   
2         169         169         220      draw   0.442463  0.205268   
3         437         318         451  home_win   0.808721  0.228151   
4          60         412         293      draw   0.192838  0.300800   
5          94         379         282      draw   0.023373  0.189668   
6         220         235         209  away_win   0.437576  0.260351   
7          97         326         302  home_win   0.500019  0.238040   
8         514         656         423  home_win   0.521809  0.232172   
9         241         439         510      draw   0.198062  0.245300   

                       
   away_win    result  
0  0.187953  home_win  
1  0.118393  home_win  
2  0.356331  home_win  
3  0.059168  home_win  
4  0.515032  away_win  
5  0.738871  away_win  
6  0.386548  home_win  
7  0.207155  home_win  
8  0.227009  home_win  
9  0.566684  away_win

# Goals prediction

### MLP

In [7]:
# fitting
from sklearn.neural_network import MLPRegressor

train = train_raw
#train  = train.drop(train[train["draw"] == True].index)

#middle: mlpG_features = ["home_team_code_sc", "away_team_code_sc", "ELO_home_sc", "ELO_away_sc", "neutral", "fGoals_home_sc", "aGoals_home_sc", "fGoals_away_sc", "aGoals_away_sc", "result_code"]
#worst: mlpG_features = ["home_team_code_sc", "away_team_code_sc", "ELO_home_sc", "ELO_away_sc", "neutral", "fGoals_p_match_home_sc", "aGoals_p_match_home_sc", "fGoals_p_match_away_sc", "aGoals_p_match_away_sc"]
mlpG_features = [   "home_team_code_sc", 
                    "away_team_code_sc", 
                    "ELO_home_sc", 
                    "ELO_away_sc", 
                    "fGoals_home_sc", 
                    "aGoals_home_sc", 
                    "fGoals_away_sc", 
                    "aGoals_away_sc", 
                    "result_code", 
                    "fGoals_p_match_home_sc", 
                    "aGoals_p_match_home_sc", 
                    "fGoals_p_match_away_sc", 
                    "aGoals_p_match_away_sc",
                    "t2t_fGoals_home_sc",
                    "t2t_fGoals_away_sc",
                    "t2t_fGoals_p_match_home_sc", 
                    "t2t_fGoals_p_match_away_sc"]
mlpG_targets = ["goal_diff", "loser_score"]

mlpG = MLPRegressor(solver="lbfgs", alpha=1e-5, hidden_layer_sizes=(100, 50, 5), random_state=1)

mlpG.fit(train[mlpG_features], train[mlpG_targets])

# make predictions
mlpG_predictions_raw = mlpG.predict(test[mlpG_features])
mlpG_predictions = pd.DataFrame(mlpG_predictions_raw, columns = mlpG_targets)

# round values and set to int
mlpG_predictions = mlpG_predictions.round().astype(int)
# if there is ever any negative value set it to 0
mlpG_predictions[mlpG_predictions < 0] = 0


# convert goal_diff and loser_score to actual score

# first attach result to prediction
mlpG_predictions.reset_index(drop=True, inplace=True)
mlpG_predictions["result"] = test["result"] 

# if home_win then home_score = loser_score+goal_diff and away_score = loser_score
mlpG_predictions.loc[mlpG_predictions["result"]  == "home_win", "home_score"] =  mlpG_predictions["loser_score"] + mlpG_predictions["goal_diff"]
mlpG_predictions.loc[mlpG_predictions["result"]  == "home_win", "away_score"] =  mlpG_predictions["loser_score"]
# if away_win vice versa
mlpG_predictions.loc[mlpG_predictions["result"]  == "away_win", "home_score"] =  mlpG_predictions["loser_score"]
mlpG_predictions.loc[mlpG_predictions["result"]  == "away_win", "away_score"] =  mlpG_predictions["loser_score"] + mlpG_predictions["goal_diff"]
# if draw then both scores are loser_score
mlpG_predictions.loc[mlpG_predictions["result"]  == "draw", "home_score"] =  mlpG_predictions["loser_score"]
mlpG_predictions.loc[mlpG_predictions["result"]  == "draw", "away_score"] =  mlpG_predictions["loser_score"]

# report
MLPG_Report_GoalDiff = metrics.classification_report(test["goal_diff"], mlpG_predictions["goal_diff"], labels=np.unique(mlpG_predictions["goal_diff"]))
print(MLPG_Report_GoalDiff)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1256
           1       0.58      0.25      0.35      1830
           2       0.27      0.60      0.37      1056
           3       0.20      0.19      0.19       553
           4       0.13      0.09      0.11       273
           5       0.12      0.05      0.07       140
           6       0.03      0.01      0.02        73
           7       0.00      0.00      0.00        43
           8       0.00      0.00      0.00        18
           9       0.11      0.05      0.07        19
          10       0.00      0.00      0.00         8
          12       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         0
          18       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         0
          29       0.00      0.00      0.00         0

   micro avg       0.47   

c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
# LOSER SCORE
MLPG_Report_LoserScore = metrics.classification_report(test["loser_score"], mlpG_predictions["loser_score"], labels=np.unique(mlpG_predictions["loser_score"]))
print(MLPG_Report_LoserScore)

In [12]:
# visualize prediction
mlpG_predictions.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
b = pd.concat([test[["home_team", "away_team", "ELO_home", "ELO_away", "neutral", "home_score", "away_score", "result"]], mlpG_predictions], axis=1, keys=["Train Data", "Prediction"])
b.head(10)

Train Data                                                             \
       home_team     away_team     ELO_home     ELO_away neutral home_score   
0          Gabon  Burkina Faso  1123.572383  1082.654529   False          1   
1         Kuwait       Lebanon  1152.753141   956.913306   False          2   
2   Sierra Leone          Togo  1124.190310  1040.581562    True          0   
3         Uganda      Tanzania  1125.830547   967.326526   False          2   
4   South Africa       Nigeria  1154.135644  1338.097182   False          0   
5       Botswana   Ivory Coast   874.408752  1262.039697   False          0   
6           Togo      Zimbabwe  1044.123140  1162.512984   False          1   
7  Guinea-Bissau        Guinea   984.975654  1088.836903   False          2   
8          Italy        Mexico  1479.033528  1380.371907   False          2   
9      Venezuela          Peru   895.606049  1096.102453   False          0   

                       Prediction                                              
  away_score    result  goal_diff loser_score    result home_score away_score  
0          1      draw          0           1      draw        1.0        1.0  
1          0  home_win          3           0  home_win        3.0        0.0  
2          0      draw          0           1      draw        1.0        1.0  
3          0  home_win          2           1  home_win        3.0        1.0  
4          0      draw          0           1      draw        1.0        1.0  
5          0      draw          0           1      draw        1.0        1.0  
6          2  away_win          2           0  away_win        0.0        2.0  
7          1  home_win          2           0  home_win        2.0        0.0  
8          0  home_win          2           0  home_win        2.0        0.0  
9          0      draw          0           1      draw        1.0        1.0

### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train = train_raw
rfG = RandomForestClassifier(n_estimators=100, min_samples_split=5, random_state=1)

rfG_features = [   "home_team_code_sc", 
                    "away_team_code_sc", 
                    "ELO_home_sc", 
                    "ELO_away_sc", 
                    "fGoals_home_sc", 
                    "aGoals_home_sc", 
                    "fGoals_away_sc", 
                    "aGoals_away_sc", 
                    "result_code", 
                    "fGoals_p_match_home_sc", 
                    "aGoals_p_match_home_sc", 
                    "fGoals_p_match_away_sc", 
                    "aGoals_p_match_away_sc",
                    "t2t_fGoals_home_sc",
                    "t2t_fGoals_away_sc",
                    "t2t_fGoals_p_match_home_sc", 
                    "t2t_fGoals_p_match_away_sc"]
rfG_targets = ["goal_diff", "loser_score"]

rfG.fit(train[rfG_features], train[rfG_targets])

# make predictions
rfG_predictions_raw =  rfG.predict(test[rfG_features])
rfG_predictions = pd.DataFrame(rfG_predictions_raw, columns = rfG_targets)

# round values and set to int
rfG_predictions = rfG_predictions.round().astype(int)
# if there is ever any negative value set it to 0
rfG_predictions[rfG_predictions < 0] = 0

# convert goal_diff and loser_score to actual score

# first attach result to prediction
rfG_predictions.reset_index(drop=True, inplace=True)
rfG_predictions["result"] = test["result"] 

# if home_win then home_score = loser_score+goal_diff and away_score = loser_score
rfG_predictions.loc[rfG_predictions["result"]  == "home_win", "home_score"] =  rfG_predictions["loser_score"] + rfG_predictions["goal_diff"]
rfG_predictions.loc[rfG_predictions["result"]  == "home_win", "away_score"] =  rfG_predictions["loser_score"]
# if away_win vice versa
rfG_predictions.loc[rfG_predictions["result"]  == "away_win", "home_score"] =  rfG_predictions["loser_score"]
rfG_predictions.loc[rfG_predictions["result"]  == "away_win", "away_score"] =  rfG_predictions["loser_score"] + rfG_predictions["goal_diff"]
# if draw then both scores are loser_score
rfG_predictions.loc[rfG_predictions["result"]  == "draw", "home_score"] =  rfG_predictions["loser_score"]
rfG_predictions.loc[rfG_predictions["result"]  == "draw", "away_score"] =  rfG_predictions["loser_score"]

# generate report
rfG_Report_GoalDiff = metrics.classification_report(test["goal_diff"], rfG_predictions["goal_diff"], labels=np.unique(rfG_predictions["goal_diff"]))
print(rfG_Report_GoalDiff)

In [ ]:
# LOSER SCORE
rfG_Report_LoserScore = metrics.classification_report(test["loser_score"], rfG_predictions["loser_score"], labels=np.unique(rfG_predictions["loser_score"]))
print(rfG_Report_LoserScore)

### XGBoost

In [17]:
import xgboost as xgb

train = train_raw
xgG = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 200)

xgG_features = [   "home_team_code_sc", 
                    "away_team_code_sc", 
                    "ELO_home_sc", 
                    "ELO_away_sc", 
                    "fGoals_home_sc", 
                    "aGoals_home_sc", 
                    "fGoals_away_sc", 
                    "aGoals_away_sc", 
                    "result_code", 
                    "fGoals_p_match_home_sc", 
                    "aGoals_p_match_home_sc", 
                    "fGoals_p_match_away_sc", 
                    "aGoals_p_match_away_sc",
                    "t2t_fGoals_home_sc",
                    "t2t_fGoals_away_sc",
                    "t2t_fGoals_p_match_home_sc", 
                    "t2t_fGoals_p_match_away_sc"]
xgG_targets = ["goal_diff", "loser_score"]

xgG.fit(train[xgG_features], train[xgG_targets])

# make predictions
xgG_predictions_raw =  xgG.predict(test[xgG_features])
xgG_predictions = pd.DataFrame(xgG_predictions_raw, columns = xgG_targets)

# round values and set to int
xgG_predictions = xgG_predictions.round().astype(int)
# if there is ever any negative value set it to 0
xgG_predictions[xgG_predictions < 0] = 0

# convert goal_diff and loser_score to actual score

# first attach result to prediction
xgG_predictions.reset_index(drop=True, inplace=True)
xgG_predictions["result"] = test["result"] 

# if home_win then home_score = loser_score+goal_diff and away_score = loser_score
xgG_predictions.loc[xgG_predictions["result"]  == "home_win", "home_score"] =  xgG_predictions["loser_score"] + xgG_predictions["goal_diff"]
xgG_predictions.loc[xgG_predictions["result"]  == "home_win", "away_score"] =  xgG_predictions["loser_score"]
# if away_win vice versa
xgG_predictions.loc[xgG_predictions["result"]  == "away_win", "home_score"] =  xgG_predictions["loser_score"]
xgG_predictions.loc[xgG_predictions["result"]  == "away_win", "away_score"] =  xgG_predictions["loser_score"] + xgG_predictions["goal_diff"]
# if draw then both scores are loser_score
xgG_predictions.loc[xgG_predictions["result"]  == "draw", "home_score"] =  xgG_predictions["loser_score"]
xgG_predictions.loc[xgG_predictions["result"]  == "draw", "away_score"] =  xgG_predictions["loser_score"]

# generate report
xgG_Report_GoalDiff = metrics.classification_report(test["goal_diff"], xgG_predictions["goal_diff"], labels=np.unique(xgG_predictions["goal_diff"]))
print(xgG_Report_GoalDiff)

c:\toolbase\python\3.11.4.2\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:11:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1216
           1       0.61      0.14      0.23      1878
           2       0.28      0.76      0.41      1066
           3       0.21      0.18      0.19       563
           4       0.17      0.09      0.12       267
           5       0.23      0.10      0.14       141
           6       0.09      0.04      0.05        53
           7       0.07      0.03      0.04        40
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00        14
          10       0.00      0.00      0.00         5
          14       0.00      0.00      0.00         0

   micro avg       0.45      0.46      0.46      5261
   macro avg       0.22      0.19      0.18      5261
weighted avg       0.54      0.46      0.42      5261



c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\toolbase\python\3.11.4.2\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# LOSER SCORE
xgG_Report_LoserScore = metrics.classification_report(test["loser_score"], xgG_predictions["loser_score"], labels=np.unique(xgG_predictions["loser_score"]))
print(xgG_Report_LoserScore)

              precision    recall  f1-score   support

           0       0.65      0.55      0.60      3012
           1       0.38      0.59      0.47      1777

   micro avg       0.51      0.57      0.54      4789
   macro avg       0.52      0.57      0.53      4789
weighted avg       0.55      0.57      0.55      4789



# TODOs

* [X] fill tournament type dict with suitable values
* [/] maybe weight the more recent games more?
* [X] plot ELO graphs and compare to ELO database for plausibility
* [_] classify draw and compare
* [X] try out XG Boost  https://xgboost.readthedocs.io/en/latest/python/index.html
* [X] Estimation: Result: Homewin/draw/Homelose, Goal difference, Goals of loser
* [_] Need to Scale the Data Before splitting into test and train -> best practice to male scaling only based on traindata
* [_] Backwards calculation of ELO values from known 2022 values
* [_] Remove all non-neutral matches from training data since they are not relvant for WM2022


Ideas for features:
* [X] Goal difference
* [X] How many goals has a team achived versus the exact opponent on average
* [X] How many wins etc.. has a team achived versus the exact opponent on average -> Probability
* [X] fGoals/aGoals average per game as feature
* [_] Use FIFA ranking (only after 1993 available -> still 26k matches) https://teams.microsoft.com/l/message/19:y6u7WdyvEHElLDbrRf470PJO9wMuxvbKqz14mIfXqCg1@thread.tacv2/1668699442716?tenantId=0ae51e19-07c8-4e4b-bb6d-648ee58410f4&groupId=99c50a52-2e25-476d-af08-f96444fdb78c&parentMessageId=1668699442716&teamName=AI%20Match%20-%20The%20Bosch%20AI%20World%20Cup%20Challenge&channelName=General&createdTime=1668699442716&allowXTenantAccess=false
* [_] Use FIFA player information (available since 2017 -> 5159 matches still)

# Observations

Overall:
* Including Goal estimations (difference, offset) in one MLP together with result makes predictions of way worse -> therfore splitting into two steps
* Droping data before 1993 leads to no performance loss
* Even droping data before 2017 leads to no noticable performance loss

Result prediction:
* RF and XG is way faster in learning than MLP, results however comparable
* removing team codes does not make predictions worse
* switching to f/aGoals_p_match does (very) slightly improve estimation compared to absolute f/aGoals
* f/aGoals etc. does only have a very small influence on result pred
* unskewed data improves performance a lot!

Goal pred
* using f/aGoals or f/aGolas_ration does not make a big difference, including both makes a slight improvement (~1% on diff and 7% on loser_goals)
* tried unskewing data based on goal_diff but doesn't help estimation in the same way as for result


# Make a prediction

In [8]:
def make_a_prediction(ht, at, neutral, algo_result, algo_goals):

    gameData= pd.DataFrame()
    # derive data
    gameData.loc[0, "neutral"] = int(neutral)
    gameData.loc[0, "home_team_code"] = team_stats.at[ht, "team_code"]
    gameData.loc[0, "away_team_code"] = team_stats.at[at, "team_code"]
    gameData.loc[0, "ELO_home"] = team_stats.at[ht, "ELO"]
    gameData.loc[0, "ELO_away"] = team_stats.at[at, "ELO"]
    gameData.loc[0, "fGoals_home"] = team_stats.at[ht, "fGoals"]
    gameData.loc[0, "aGoals_home"] = team_stats.at[ht, "aGoals"]
    gameData.loc[0, "fGoals_away"] = team_stats.at[at, "fGoals"]
    gameData.loc[0, "aGoals_away"] = team_stats.at[at, "aGoals"]
    gameData.loc[0, "fGoals_p_match_home"] = team_stats.at[ht, "fGoals_p_match"]
    gameData.loc[0, "aGoals_p_match_home"] = team_stats.at[ht, "aGoals_p_match"]
    gameData.loc[0, "fGoals_p_match_away"] = team_stats.at[at, "fGoals_p_match"]
    gameData.loc[0, "aGoals_p_match_away"] = team_stats.at[at, "aGoals_p_match"]
    # add t2t data
    gameData.loc[0, [   "t2t_fGoals_home",
                        "t2t_fGoals_away",
                        "t2t_fGoals_p_match_home",
                        "t2t_fGoals_p_match_away",
                        "t2t_num_wins_home",
                        "t2t_num_wins_away",
                        "t2t_num_draw",
                        "t2t_p_wins_home",
                        "t2t_p_wins_away",
                        "t2t_p_draw"]] = get_t2t_data(ht, at, "2033-01-01")

    # scale
    gameData[[  "fGoals_home_sc", 
                "aGoals_home_sc", 
                "fGoals_away_sc", 
                "aGoals_away_sc",
                "t2t_fGoals_home_sc",
                "t2t_fGoals_away_sc" ]] = goal_scaler.transform(gameData[["fGoals_home", "aGoals_home", "fGoals_away", "aGoals_away", "t2t_fGoals_home", "t2t_fGoals_away"]])

    gameData[[  "fGoals_p_match_home_sc", 
                "aGoals_p_match_home_sc", 
                "fGoals_p_match_away_sc", 
                "aGoals_p_match_away_sc",
                "t2t_fGoals_p_match_home_sc",
                "t2t_fGoals_p_match_away_sc" ]] = goal_pm_scaler.transform(gameData[["fGoals_p_match_home", "aGoals_p_match_home", "fGoals_p_match_away", "aGoals_p_match_away","t2t_fGoals_p_match_home", "t2t_fGoals_p_match_away"]])

    gameData[[  "ELO_home_sc", 
                "ELO_away_sc"]] = ELO_scaler.transform(gameData[["ELO_home", "ELO_away"]])

    # gameData[[   "tournament_code_sc" ]] = tournament_scaler.transform(gameData[["tournament_code" ]])

    gameData[[  "home_team_code_sc", 
                "away_team_code_sc"]] = team_scaler.transform(gameData[["home_team_code", "away_team_code"]])

    gameData[[  "t2t_num_wins_home_sc", 
                "t2t_num_wins_away_sc",
                "t2t_num_draw_sc" ]] = num_win_scaler.transform(gameData[["t2t_num_wins_home", "t2t_num_wins_away", "t2t_num_draw"]])

    ######### GAME RESULT PREDICTION ################

    # algo_dependend start
    if (algo_result == "rf"):
        game_redictions_raw =  rf.predict(gameData[rf_features])
        game_predictions = pd.DataFrame(game_redictions_raw, columns = rf_targets)
    elif (algo_result == "xg"):
        game_redictions_raw =  xg_reg.predict(gameData[xg_features])
        game_predictions = pd.DataFrame(game_redictions_raw, columns = xg_targets)
        # translate result probs into result
        game_predictions["result"] = game_predictions[["home_win", "away_win", "draw"]].idxmax(axis=1) # needed 
    elif (algo_result == "mlp"):
        game_redictions_raw =  mlp.predict(gameData[mlp_features])
        game_predictions = pd.DataFrame(game_redictions_raw, columns = mlp_targets)
        # translate result probs into result
        game_predictions["result"] = game_predictions[["home_win", "away_win", "draw"]].idxmax(axis=1) # needed 
    else:
        print("ERROR: algo_result unknown")
    
    # algo_dependend end

    print("--------------- Result  --------------------")
    # print("Prediction of game result: " + xg_game_ppredictions["result"][0])
    if (game_predictions["result"][0] == "home_win"):
        print(ht + " wins!")
        gameData["result_code"] = 2  # magic number,how to retrive?
    elif (game_predictions["result"][0] == "away_win"):
        print(at + " wins!")
        gameData["result_code"] = 0  # magic number,how to retrive?
    else:
        print("draw!")
        gameData["result_code"] = 1  # magic number,how to retrive?


    ######### GOAL PREDICTION ################

    # add result from previus step into input vector 
    gameData["result_code"] = game_predictions["result"].astype("category").cat.codes

    # algo_dependend start
    if (algo_goals == "mlp"):
        game_goals_prediction_raw = mlpG.predict(gameData[mlpG_features])
        # postprocess
        game_goals_prediction = pd.DataFrame(game_goals_prediction_raw, columns = mlpG_targets)
    elif (algo_goals == "xg"):
        game_goals_prediction_raw = xgG_reg.predict(gameData[xgG_features])
        # postprocess
        game_goals_prediction = pd.DataFrame(game_goals_prediction_raw, columns = xgG_targets)
    elif (algo_goals == "rf"):
        game_goals_prediction_raw =  rfG.predict(gameData[rfG_features])
        game_goals_prediction = pd.DataFrame(game_goals_prediction_raw, columns = rfG_targets)
    else:
        print("ERROR: algo_goals unknown")
    # algo_dependend end

    
    # round values and set to int
    game_goals_prediction = game_goals_prediction.round().astype(int)
    # if there is ever any negative value set it to 0
    game_goals_prediction[game_goals_prediction < 0] = 0

    if (game_predictions["result"][0] == "home_win"):
        goals_away = int(game_goals_prediction["loser_score"])
        goals_home = int(game_goals_prediction["loser_score"] + game_goals_prediction["goal_diff"])
    elif (game_predictions["result"][0] == "away_win"):
        goals_away = int(game_goals_prediction["loser_score"] + game_goals_prediction["goal_diff"])
        goals_home = int(game_goals_prediction["loser_score"])
    else:
        goals_away = int(game_goals_prediction["loser_score"])
        goals_home = int(game_goals_prediction["loser_score"])


    print("Goals", ht, ":", goals_home)
    print("Goals", at, ":", goals_away)
    print("\n")
    print("--------------- Input Data --------------------")
    print("ELO", ht, ":", gameData["ELO_home"][0])
    print("ELO", at, ":", gameData["ELO_away"][0])
    print("t2t_fGoals_p_match", ht, ":", gameData["t2t_fGoals_p_match_home"][0])
    print("t2t_fGoals_p_match", at, ":", gameData["t2t_fGoals_p_match_away"][0])
    print("t2t_num_wins", ht, ":", gameData["t2t_num_wins_home"][0])
    print("t2t_num_wins", at, ":", gameData["t2t_num_wins_away"][0])
    print("t2t_num_draw :", gameData["t2t_num_draw"][0])

    gameData.head()

In [10]:
####### USER INPUT ##########
ht = "Spain"
at = "England"
neutral = True
####### USER INPUT END ##########

make_a_prediction(ht, at, neutral, algo_result="xg", algo_goals="mlp")
# make_a_prediction(at, ht, neutral, algo_result="xg", algo_goals="mlp")



--------------- Result  --------------------
England wins!
Goals Spain : 1
Goals England : 2


--------------- Input Data --------------------
ELO Spain : 1633.7069709552934
ELO England : 1649.4202101649344
t2t_fGoals_p_match Spain : 1.1851851851851851
t2t_fGoals_p_match England : 1.6666666666666667
t2t_num_wins Spain : 10.0
t2t_num_wins England : 13.0
t2t_num_draw : 4.0


C:\Users\dar7si\AppData\Local\Temp\ipykernel_30148\3103502340.py:122: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  goals_away = int(game_goals_prediction["loser_score"] + game_goals_prediction["goal_diff"])
C:\Users\dar7si\AppData\Local\Temp\ipykernel_30148\3103502340.py:123: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  goals_home = int(game_goals_prediction["loser_score"])
